In [6]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np 
device = tf.device("gpu:0")
# Step 1: Dataset Preparation
# Set the paths to your dataset
train_data_dir =   './Train/'
test_data_dir =   'Val/'

# Step 2: Data Preprocessing
image_size = (224, 224)  # Adjust the size as per your requirements

# Data augmentation for training set (optional)

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=10,  # Example of augmentation parameters
    zoom_range=0.1,
    horizontal_flip=True
)


# Data normalization for testing set
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical'
)
# Step 5: Check the class labels
class_labels = list(train_generator.class_indices.keys())
num_classes = len(class_labels)

# Step 6: Check the class imbalance
class_counts = np.sum(train_generator.labels, axis=0)

# Step 7: Print dataset statistics
print("Number of classes:", num_classes)
print("Class labels:", class_labels)
print("Class counts:", class_counts)




Found 5354 images belonging to 3 classes.
Found 1316 images belonging to 3 classes.
Number of classes: 3
Class labels: ['Strawberry___Leaf_scorch', 'Strawberry___healthy', 'nonValid']
Class counts: 5336


In [7]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.layers import Dropout
l1_weight = 0.001  # L1 regularization weight
l2_weight = 0.001  # L2 regularization weight
dropout_rate = 0.5  # Dropout rate


# Step 3: Model Creation and Fine-tuning
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224,224, 3))

for layer in base_model.layers:  # Fine-tune the last n layers
    layer.trainable = False

x = base_model.output

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)

# Add more convolutional layers if desired
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2(l1_weight, l2_weight))(x)
x = layers.Dropout(0.5)(x)

x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

x = Dropout(dropout_rate)(x)  # Adding a Dropout layer with the specified dropout rate
predictions = layers.Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
# ...
# After creating the final model

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1)
checkpoint = ModelCheckpoint('STRAWBERRYBESTMODEL.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)


In [9]:
with tf.device("/GPU:0"):
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        steps_per_epoch=train_generator.n // train_generator.batch_size,
        validation_steps=validation_generator.n // validation_generator.batch_size,
        epochs=10,
        callbacks=[reduce_lr, checkpoint, early_stop]
    )



Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[32,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_1/block1_conv1/Relu (defined at C:\Users\Marwan\AppData\Local\Temp\ipykernel_30340\53586518.py:2) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_3228]

Function call stack:
train_function
